<a href="https://colab.research.google.com/github/Alyssasorensen/sqlite_database_operations/blob/main/sqlite_database_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3 Homework Assignment: Introduction to Databases with SQLite



### Loading Packages

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

### Loading Datasets

In [3]:
sbm = pd.read_csv('https://raw.githubusercontent.com/Alyssasorensen/sqlite_database_operations/main/datasets/113243405_StonyBrookUniversityHospital_standardcharges.csv')

In [4]:
sbm

,Code,Description,Type,Package/Line_Level,Gross charge,Discounted cash price,De-identified min contracted rate,De-identified max contracted rate,Derived contracted rate,1199-Commercial other,...,Optum-Commercial other,Oxford-Commercial other,Oxford-Commercial HMO/POS,Tricare-Commercial other,United Healthcare-Commercial other,United Healthcare-Medicare Advantage HMO,United Healthcare-Commercial HMO/POS,United Healthcare-Medicaid HMO,United Healthcare-Commercial PPO/Open Access,Veteran Family-Commercial other
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...",Outpatient,Line,718.67,718.67,2155.50,2155.50,2155.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
2,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Package,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
3,10006,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,728.54,728.54,58.55,1761.25,909.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,2823.01,2823.01,446.28,6017.59,4219.31,4592.0,...,NaN,NaN,NaN,NaN,5127.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,30981203,R&B SICU 09W1 ICR PT-Z,NaN,NaN,12090.00,12090.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7706,30993018,R&B TICR 09W2 - PT-I,NaN,NaN,13590.00,13590.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7707,33500000,R&B REV PICU 11S2 PT-X,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7708,30333058,ROPH HOSP.OBSERVATION PT-T,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
nyu = pd.read_excel('NYUstandardcharges.xlsx')

In [8]:
nyu

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,NYU Langone Brooklyn,NYU Langone Brooklyn.1,NYU Langone Brooklyn.2,NYU Langone Brooklyn.3,NYU Langone Brooklyn.4,...,NYU Langone Brooklyn.391,NYU Langone Brooklyn.392,NYU Langone Brooklyn.393,NYU Langone Brooklyn.394,NYU Langone Brooklyn.395,NYU Langone Brooklyn.396,NYU Langone Brooklyn.397,NYU Langone Brooklyn.398,Unnamed: 404,Unnamed: 405
0,NaN,NaN,NaN,NaN,NaN,1005,1006,1439,1184,1185,...,3301,1805,1806,1807,1808,1802,2236,1088,NaN,NaN
1,Identifier Code,Billing Code,Identifier Description,NYU Langone Gross Charges,NYU Langone Discounted Cash Price,AETNA HMO,AETNA INDEMNITY,BCBS GHI COMPOSITE,BCBS HMO,BCBS INDEMNITY,...,HEALTHFIRST ESSENTIAL PLAN 1&2,HEALTHFIRST GOLD LEAF EXCHANGE,HEALTHFIRST BRONZE LEAF EXCHANGE,HEALTHFIRST GREEN LEAF EXCHANGE,HEALTHFIRST PLATINUM LEAF EXCHANGE,HEALTHFIRST SILVER LEAF EXCHANGE,"HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO",HEALTHNET PPO,De-identified Minimum,De-identified Maximum
2,2023696,19120,HC EXC CYST/ABERRANT BREAST TISSUE OPEN 1/> LE...,7750.52,1472.6,9190.33,7750.52,10255,10255,10255,...,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,6277.92,776.73,22768
3,2023358,43235,HC UGI DIAGNOSTIC,3344.09,635.38,3670.36,3344.09,5901,5901,5901,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,2708.71,248.4,8498
4,2023360,43239,HC UGI W BX. SGL/MULTIPLE,1464.39,278.23,4699.3,1464.39,5901,5901,5901,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,1186.16,276.43,8195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,6071446,C1771,SINGLE PRECISE MINIARC,21746.5,4131.83,22420.64,21746.5,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,16744.8,16744.8,993.27,22420.64
7416,6079443,C1771,SYS OBTRYX II PRECISIONBLUE DESIGN,23685.06,4500.16,24419.29,23685.06,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,18237.49,18237.49,1081.82,24419.29
7417,6080126,C1771,SLING URETHRAL FIT ADVANTAGE,22200.8,4218.15,22889.02,22200.8,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,17094.61,17094.61,169.51,22889.02
7418,6081272,C1771,SYS SLING RETROPUBIC RETROARC,16761.94,3184.77,17281.56,16761.94,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,12906.69,12906.69,169.51,17281.56


## 1. Data Exploration and Analysis:

In [9]:
print("Dataset 1 Overview:")
print(sbm.head())
print("Dataset 2 Overview")
print(nyu.head)

Dataset 1 Overview:
    Code                                        Description        Type  \
0  10004  FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...  Outpatient   
1  10005  FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...  Outpatient   
2  10005  FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...  Outpatient   
3  10006  FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...  Outpatient   
4  10009  FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...  Outpatient   

  Package/Line_Level  Gross charge  Discounted cash price  \
0               Line        718.67                 718.67   
1               Line       2061.25                2061.25   
2            Package       2061.25                2061.25   
3               Line        728.54                 728.54   
4               Line       2823.01                2823.01   

   De-identified min contracted rate  De-identified max contracted rate  \
0                            2155.50                            2155.50   
1           

In [10]:
print("Summary Statistics for Dataset 1:")
print(sbm.describe())
print("Summary Statistics for Dataset 2:")
print(nyu.describe())

Summary Statistics for Dataset 1:
       Gross charge  Discounted cash price  De-identified min contracted rate  \
count  7.710000e+03           7.710000e+03                        6254.000000   
mean   2.274832e+04           2.274832e+04                        4090.693876   
std    7.470570e+04           7.470570e+04                       10576.213503   
min    3.000000e-02           3.000000e-02                           0.010000   
25%    1.065895e+03           1.065895e+03                          96.520000   
50%    4.745295e+03           4.745295e+03                        1108.860000   
75%    1.103881e+04           1.103881e+04                        4309.532500   
max    1.857952e+06           1.857952e+06                      327869.690000   

       De-identified max contracted rate  Derived contracted rate  \
count                       6.254000e+03              6254.000000   
mean                        1.003579e+04              6315.587725   
std                         2

### Cleaning Data and Handling Missing Values

In [11]:
def clean_column_names(sbm):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    sbm.columns = [clean_name(col) for col in sbm.columns]
    return sbm

# Apply the clean_value function to all columns
sbm = clean_column_names(sbm)

sbm

,code,description,type,packagelinelevel,grosscharge,discountedcashprice,deidentifiedmincontractedrate,deidentifiedmaxcontractedrate,derivedcontractedrate,1199commercialother,...,optumcommercialother,oxfordcommercialother,oxfordcommercialhmopos,tricarecommercialother,unitedhealthcarecommercialother,unitedhealthcaremedicareadvantagehmo,unitedhealthcarecommercialhmopos,unitedhealthcaremedicaidhmo,unitedhealthcarecommercialppoopenaccess,veteranfamilycommercialother
0,10004,"FINE NEEDLE ASPIRATION BIOPSY, EACH ADDITIONAL...",Outpatient,Line,718.67,718.67,2155.50,2155.50,2155.50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
2,10005,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Package,2061.25,2061.25,87.02,4782.00,2308.44,2411.0,...,NaN,4782.0,NaN,NaN,3845.25,NaN,NaN,658.94,NaN,NaN
3,10006,FINE NEEDLE ASPIRATION BIOPSY USING ULTRASOUND...,Outpatient,Line,728.54,728.54,58.55,1761.25,909.90,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10009,FINE NEEDLE ASPIRATION BIOPSY OF GROWTH USING ...,Outpatient,Line,2823.01,2823.01,446.28,6017.59,4219.31,4592.0,...,NaN,NaN,NaN,NaN,5127.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,30981203,R&B SICU 09W1 ICR PT-Z,NaN,NaN,12090.00,12090.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7706,30993018,R&B TICR 09W2 - PT-I,NaN,NaN,13590.00,13590.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7707,33500000,R&B REV PICU 11S2 PT-X,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7708,30333058,ROPH HOSP.OBSERVATION PT-T,NaN,NaN,8165.00,8165.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def clean_column_names(nyu):
    # Define a helper function to clean column names
    def clean_name(name):
        cleaned_name = re.sub(r'[^a-zA-Z0-9]', '', name)
        return cleaned_name.lower()

    # Rename columns using the helper function
    # This is using a list comprehend - e.g., we have a list to the right of the equals sign,
    # and inside the list, we are applying our function, for every col (or X) that exists in df.columns
    nyu.columns = [clean_name(col) for col in nyu.columns]
    return nyu

# Apply the clean_value function to all columns
nyu = clean_column_names(nyu)

nyu

,unnamed0,unnamed1,unnamed2,unnamed3,unnamed4,nyulangonebrooklyn,nyulangonebrooklyn1,nyulangonebrooklyn2,nyulangonebrooklyn3,nyulangonebrooklyn4,...,nyulangonebrooklyn391,nyulangonebrooklyn392,nyulangonebrooklyn393,nyulangonebrooklyn394,nyulangonebrooklyn395,nyulangonebrooklyn396,nyulangonebrooklyn397,nyulangonebrooklyn398,unnamed404,unnamed405
0,NaN,NaN,NaN,NaN,NaN,1005,1006,1439,1184,1185,...,3301,1805,1806,1807,1808,1802,2236,1088,NaN,NaN
1,Identifier Code,Billing Code,Identifier Description,NYU Langone Gross Charges,NYU Langone Discounted Cash Price,AETNA HMO,AETNA INDEMNITY,BCBS GHI COMPOSITE,BCBS HMO,BCBS INDEMNITY,...,HEALTHFIRST ESSENTIAL PLAN 1&2,HEALTHFIRST GOLD LEAF EXCHANGE,HEALTHFIRST BRONZE LEAF EXCHANGE,HEALTHFIRST GREEN LEAF EXCHANGE,HEALTHFIRST PLATINUM LEAF EXCHANGE,HEALTHFIRST SILVER LEAF EXCHANGE,"HEALTHFIRST BRONZE TOTAL, PRO, PLUS EPO",HEALTHNET PPO,De-identified Minimum,De-identified Maximum
2,2023696,19120,HC EXC CYST/ABERRANT BREAST TISSUE OPEN 1/> LE...,7750.52,1472.6,9190.33,7750.52,10255,10255,10255,...,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,1792.32,6277.92,776.73,22768
3,2023358,43235,HC UGI DIAGNOSTIC,3344.09,635.38,3670.36,3344.09,5901,5901,5901,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,2708.71,248.4,8498
4,2023360,43239,HC UGI W BX. SGL/MULTIPLE,1464.39,278.23,4699.3,1464.39,5901,5901,5901,...,975.42,975.42,975.42,975.42,975.42,975.42,975.42,1186.16,276.43,8195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,6071446,C1771,SINGLE PRECISE MINIARC,21746.5,4131.83,22420.64,21746.5,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,16744.8,16744.8,993.27,22420.64
7416,6079443,C1771,SYS OBTRYX II PRECISIONBLUE DESIGN,23685.06,4500.16,24419.29,23685.06,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,18237.49,18237.49,1081.82,24419.29
7417,6080126,C1771,SLING URETHRAL FIT ADVANTAGE,22200.8,4218.15,22889.02,22200.8,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,17094.61,17094.61,169.51,22889.02
7418,6081272,C1771,SYS SLING RETROPUBIC RETROARC,16761.94,3184.77,17281.56,16761.94,-1,-1,-1,...,NSP,NSP,NSP,NSP,NSP,NSP,12906.69,12906.69,169.51,17281.56


In [13]:
print("Missing Values in Dataset 1:")
print(sbm.isnull().sum())

Missing Values in Dataset 1:
code                                          0
description                                   0
type                                        783
packagelinelevel                            783
grosscharge                                   0
discountedcashprice                           0
deidentifiedmincontractedrate              1456
deidentifiedmaxcontractedrate              1456
derivedcontractedrate                      1456
1199commercialother                        6696
aetnamedicareadvantagehmo                  6604
aetnacommercialhmopos                      5883
aetnacommercialppoopenaccess               6098
aetnacommercialother                       7582
empirehealthcommercialother                4650
empirehealthcommercialppoopenaccess        5308
bluecrossblueshieldcommercialother         6456
beaconhealthcommercialother                7707
carelonhealthcommercialother               7707
cignacommercialppoopenaccess               5744
cignacommer

In [14]:
print("Missing Values in Dataset 2:")
print(nyu.isnull().sum())

Missing Values in Dataset 2:
unnamed0                 1
unnamed1                 1
unnamed2                 1
unnamed3                 1
unnamed4                 1
                        ..
nyulangonebrooklyn396    0
nyulangonebrooklyn397    0
nyulangonebrooklyn398    0
unnamed404               1
unnamed405               1
Length: 406, dtype: int64


### Basic Statistics SBM

In [15]:
# Example: Frequency counts for a categorical column 'Description' in Dataset 1
print("Frequency Counts for 'Description' in Dataset 1:")
print(sbm['description'].value_counts())

Frequency Counts for 'Description' in Dataset 1:
R&B MED 05CA SEMI-PRIVATE/2                                                                   3
R&B SHA 05PS/11PS SEMI-PRIVATE                                                                3
DESTRUCTION OF FIRST DIVISION OF FACE NERVE BRANCH                                            2
INJECTION OF CHEMICAL FOR PARALYSIS OF SALIVARY GLANDS ON BOTH SIDES OF MOUTH                 2
INJECTION OF CHEMICAL FOR PARALYSIS OF FACIAL AND NECK NERVE MUSCLES ON BOTH SIDES OF FACE    2
                                                                                             ..
ANCHOR/SCREW FOR OPPOSING BONE-TO-BONE OR SOFT TISSUE-TO-BONE (IMPLANTABLE)                   1
BRACHYTHERAPY NEEDLE                                                                          1
CARDIOVERTER-DEFIBRILLATOR, DUAL CHAMBER (IMPLANTABLE)                                        1
CARDIOVERTER-DEFIBRILLATOR, SINGLE CHAMBER (IMPLANTABLE)                               

In [70]:
# List of numerical columns to calculate central tendency measures for
numerical_columns = [
    "grosscharge",
    "discountedcashprice",
    "deidentifiedmincontractedrate",
    "deidentifiedmaxcontractedrate",
    "derivedcontractedrate",
    "1199commercialother",
    "aetnamedicareadvantagehmo",
    "aetnacommercialhmopos",
    "aetnacommercialppoopenaccess",
    "aetnacommercialother",
    "empirehealthcommercialother",
    "empirehealthcommercialppoopenaccess",
    "bluecrossblueshieldcommercialother",
    "beaconhealthcommercialother",
    "carelonhealthcommercialother",
    "cignacommercialppoopenaccess",
    "cignacommercialother",
    "cignacommercialhmopos",
    "ehfacetcommercialother",
    "emblemhealthcommercialppoopenaccess",
    "emblemhealthcommercialother",
    "emblemhealthcommercialhmopos",
    "emblemhealthmedicaidhmo",
    "emblemhealthmedicareadvantagehmo",
    "empirehealthcommercialhmopos",
]

# Calculate mean, median, and mode for each numerical column
central_tendency_measures = {}
for column in numerical_columns:
    mean = sbm[column].mean()
    median = sbm[column].median()
    mode = sbm[column].mode()
    central_tendency_measures[column] = {
        "Mean": mean,
        "Median": median,
        "Mode": mode,
    }

# Display the central tendency measures
central_tendency_df = pd.DataFrame(central_tendency_measures)
print(central_tendency_df)


                                          grosscharge  \
Mean                                     22748.319449   
Median                                       4745.295   
Mode    0    8165.0
Name: grosscharge, dtype: float64   

                                      discountedcashprice  \
Mean                                         22748.319449   
Median                                           4745.295   
Mode    0    8165.0
Name: discountedcashprice, dtype: ...   

                            deidentifiedmincontractedrate  \
Mean                                          4090.693876   
Median                                            1108.86   
Mode    0    6050.0
Name: deidentifiedmincontractedrat...   

                            deidentifiedmaxcontractedrate  \
Mean                                         10035.793743   
Median                                             4311.0   
Mode    0    6050.0
Name: deidentifiedmaxcontractedrat...   

                                    

/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [71]:
# List of numerical columns to calculate central tendency measures for
numerical_columns = [
    "empirehealthmedicareadvantagehmo",
    "empirehealthmedicaidhmo",
    "evernorthcommercialother",
    "fideliscommercialother",
    "fidelismedicareadvantagehmo",
    "fidelismedicaidhmo",
    "ghicommercialother",
    "healthfirstcommercialother",
    "healthfirstmedicareadvantagehmo",
    "healthfirstmedicaidhmo",
    "healthplushpmedicaidhmo",
    "healthplushpcommercialother",
    "healthplushpmedicareadvantagehmo",
    "humanacommercialother",
    "humanacommercialhmopos",
    "humanacommercialppoopenaccess",
    "meritainhealthcommercialother",
    "molinacommercialother",
    "optumcommercialother",
    "oxfordcommercialother",
    "oxfordcommercialhmopos",
    "tricarecommercialother",
    "unitedhealthcarecommercialother",
    "unitedhealthcaremedicareadvantagehmo",
    "unitedhealthcarecommercialhmopos",
    "unitedhealthcaremedicaidhmo",
    "unitedhealthcarecommercialppoopenaccess",
    "veteranfamilycommercialother",
]
# Calculate mean, median, and mode for each numerical column
central_tendency_measures = {}
for column in numerical_columns:
    mean = sbm[column].mean()
    median = sbm[column].median()
    mode = sbm[column].mode()
    central_tendency_measures[column] = {
        "Mean": mean,
        "Median": median,
        "Mode": mode,
    }

# Display the central tendency measures
central_tendency_df = pd.DataFrame(central_tendency_measures)
print(central_tendency_df)

                         empirehealthmedicareadvantagehmo  \
Mean                                          5846.313678   
Median                                             570.35   
Mode    0    125.07
Name: empirehealthmedicareadvantag...   

                                  empirehealthmedicaidhmo  \
Mean                                              6943.79   
Median                                            6943.79   
Mode    0    6122.75
1    7764.83
Name: empirehealthme...   

                                 evernorthcommercialother  \
Mean                                              133.245   
Median                                            138.105   
Mode    0     55.92
1    102.98
2    106.39
3    136.4...   

                                   fideliscommercialother  \
Mean                                           2462.66825   
Median                                              200.0   
Mode    0    300.0
Name: fideliscommercialother, dtype...   

                    

/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [72]:
sbm_grosscharge_mean = sbm['grosscharge'].mean()
sbm_grosscharge_median = sbm['grosscharge'].median()
sbm_grosscharge_mode = sbm['grosscharge'].mode().values[0]

In [73]:
sbm_grosscharge_range = sbm['grosscharge'].max() - sbm['grosscharge'].min()
sbm_grosscharge_var = sbm['grosscharge'].var()
sbm_grosscharge_std= sbm['grosscharge'].std()
sbm_grosscharge_iqr = sbm['grosscharge'].quantile(0.75) - sbm['grosscharge'].quantile(0.25)

In [74]:
print("Measures of Central Tendency:")
print(f"Mean: {sbm_grosscharge_mean}")
print(f"Median: {sbm_grosscharge_median}")
print(f"Mode: {sbm_grosscharge_mode}")
print("\nMeasures of Spread:")
print(f"Range: {sbm_grosscharge_range}")
print(f"Variance: {sbm_grosscharge_var}")
print(f"Standard Deviation: {sbm_grosscharge_std}")
print(f"IQR (Interquartile Range): {sbm_grosscharge_iqr}")

Measures of Central Tendency:
Mean: 22748.319448767834
Median: 4745.295
Mode: 8165.0

Measures of Spread:
Range: 1857952.41
Variance: 5580942100.484088
Standard Deviation: 74705.70326611007
IQR (Interquartile Range): 9972.9175


In [20]:
sbm_aetnacommercialhmopos_mean = sbm['aetnacommercialhmopos'].mean()


sbm_aetnacommercialhmopos_median = sbm['aetnacommercialhmopos'].median()


sbm_aetnacommercialhmopos_mode = sbm['aetnacommercialhmopos'].mode().values[0]


sbm_aetnacommercialhmopos_range = sbm['aetnacommercialhmopos'].max() - sbm['aetnacommercialhmopos'].min()
sbm_aetnacommercialhmopos_var = sbm['aetnacommercialhmopos'].var()
sbm_aetnacommercialhmopos_std= sbm['aetnacommercialhmopos'].std()
sbm_aetnacommercialhmopos_iqr = sbm['aetnacommercialhmopos'].quantile(0.75) - sbm['aetnacommercialhmopos'].quantile(0.25)


print("Measures of Central Tendency:")
print(f"Mean: {sbm_aetnacommercialhmopos_mean}")
print(f"Median: {sbm_aetnacommercialhmopos_median}")
print(f"Mode: {sbm_aetnacommercialhmopos_mode}")
print("\nMeasures of Spread:")
print(f"Range: {sbm_aetnacommercialhmopos_range}")
print(f"Variance: {sbm_aetnacommercialhmopos_var}")
print(f"Standard Deviation: {sbm_aetnacommercialhmopos_std}")
print(f"IQR (Interquartile Range): {sbm_aetnacommercialhmopos_iqr}")

Measures of Central Tendency:
Mean: 4320.4182484948005
Median: 2153.0
Mode: 4822.0

Measures of Spread:
Range: 99999.97
Variance: 76012890.5927689
Standard Deviation: 8718.537181934185
IQR (Interquartile Range): 4544.1


In [21]:
print (sbm['code'].value_counts())

73521       2
64600       2
64454       2
64461       2
64483       2
           ..
C1727       1
C1728       1
C1729       1
C1730       1
30000004    1
Name: code, Length: 5474, dtype: int64


In [22]:
print (sbm['description'].value_counts())

R&B MED 05CA SEMI-PRIVATE/2                                                                   3
R&B SHA 05PS/11PS SEMI-PRIVATE                                                                3
DESTRUCTION OF FIRST DIVISION OF FACE NERVE BRANCH                                            2
INJECTION OF CHEMICAL FOR PARALYSIS OF SALIVARY GLANDS ON BOTH SIDES OF MOUTH                 2
INJECTION OF CHEMICAL FOR PARALYSIS OF FACIAL AND NECK NERVE MUSCLES ON BOTH SIDES OF FACE    2
                                                                                             ..
ANCHOR/SCREW FOR OPPOSING BONE-TO-BONE OR SOFT TISSUE-TO-BONE (IMPLANTABLE)                   1
BRACHYTHERAPY NEEDLE                                                                          1
CARDIOVERTER-DEFIBRILLATOR, DUAL CHAMBER (IMPLANTABLE)                                        1
CARDIOVERTER-DEFIBRILLATOR, SINGLE CHAMBER (IMPLANTABLE)                                      1
SEMI-PRIVATE                            

In [96]:
print (sbm['type'].value_counts())

Outpatient    6041
Inpatient      886
Name: type, dtype: int64


In [23]:
print (sbm['packagelinelevel'].value_counts())

Line       4691
Package    2236
Name: packagelinelevel, dtype: int64


### Basic Statistics for NYU

In [16]:
# Example: Frequency counts for a categorical column 'nyulangonebrooklyn398' in Dataset 2
print("Frequency Counts for 'nyulangonebrooklyn398' in Dataset 2:")
print(nyu['nyulangonebrooklyn398'].value_counts())

Frequency Counts for 'nyulangonebrooklyn398' in Dataset 2:
-1.00         308
 1981.63      196
 990.82       159
 1512.30      152
 1877.34      123
             ... 
 32196.33       1
 4557.76        1
 456.92         1
 2133.59        1
 135870.37      1
Name: nyulangonebrooklyn398, Length: 2405, dtype: int64


## 2. SQLite Database Operations:

In [16]:
## Create a new blank database using sqlite3

conn = sqlite3.connect('costs.db')
c = conn.cursor()

In [17]:
## Create a new table called 'stonybrook' and load in the data from stonybrook_modified
## Manually create the schema based on the data types in stonybrook_modified

c.execute('''CREATE TABLE stonybrook
                (
                    [hospital_name] text,
                    [insurance_type] text,
                    [code] text,
                    [code_description] text,
                    [cost_negotiated] real,
                    [cost_minimum] real,
                    [cost_maximum] real
                )''')

conn.commit()

In [18]:
## Now print out the table names from the database to make sure it worked

c.execute('''SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;''')

for value in c.fetchall():
    print(value)

('stonybrook',)


In [19]:
## Now perform a Select * on the table to make sure it worked and it returns no values

c.execute('''SELECT * FROM stonybrook''')
print(c.fetchall())

[]


In [20]:
## Now perform the same query in pandas to make sure it worked

pd.read_sql('''SELECT * FROM stonybrook''', conn)

,hospital_name,insurance_type,code,code_description,cost_negotiated,cost_minimum,cost_maximum
